## Setup

In [ ]:
import os

import arrow
import sqlalchemy
import psycopg2

from prodiguer.db import pgres as db

# Set timeslice to 12 months - returns 132480 jobs.
_TIMESLICE = arrow.utcnow().replace(days=-365)

# Start SqlAlchemy session.
db.session.start()

# Set SqlAlchemy query targets.
j = db.types.Job
s = db.types.Simulation

# Start psycopg2 session.
db_conn = psycopg2.connect(
    database=db.constants.PRODIGUER_DB_NAME,
    user=db.constants.PRODIGUER_DB_USER,
    host=os.getenv("PRODIGUER_DB_PGRES_HOST").split(":")[0]
    )

## SqlAlchemy with ORM mapper :: 4.89 seconds

In [ ]:
# Code is simple but slow for large queries due to ORM overhead.
qry = db.session.query(j)
qry = qry.join(s, j.simulation_uid==s.uid)
qry = qry.filter(j.execution_start_date != None)
qry = qry.filter(s.execution_start_date != None)
qry = qry.filter(s.is_obsolete == False)
qry = qry.filter(s.execution_start_date >= _TIMESLICE)

jobs = qry.all()

## SqlAlchemy without ORM mapper :: 1.42 seconds

In [ ]:
# Code is simple and fast.
qry = db.sa_session.query(
    j.execution_end_date,
    j.execution_start_date,
    j.is_compute_end,
    j.is_error,
    j.job_uid,
    j.simulation_uid,
    j.typeof
    )
qry = qry.join(s, j.simulation_uid == s.uid)
qry = qry.filter(j.execution_start_date != None)
qry = qry.filter(s.execution_start_date != None)
qry = qry.filter(s.is_obsolete == False)
qry = qry.filter(s.execution_start_date >= _TIMESLICE)

jobs = qry.all()

## psycopg2 direct :: 1.23 seconds

In [ ]:
# Code is messy but fast.
SQL = """SELECT
    to_char(j.execution_end_date, 'YYYY-MM-DD HH24:MI:ss.US'),
    to_char(j.execution_start_date, 'YYYY-MM-DD HH24:MI:ss.US'),
    j.is_compute_end,
    j.is_error,
    j.job_uid,
    j.simulation_uid,
    j.typeof
FROM
    monitoring.tbl_job as j
JOIN
    monitoring.tbl_simulation as s ON j.simulation_uid = s.uid
WHERE
    j.execution_start_date IS NOT NULL AND
    s.execution_start_date IS NOT NULL AND
    s.is_obsolete = false AND
    s.execution_start_date >= '{}'
ORDER BY
    j.execution_start_date;
""".format(_TIMESLICE)

cur = db_conn.cursor()
cur.execute(SQL)
jobs = = cur.fetchall()

## Cleanup

In [ ]:
# Close SqlAlchmey session.
db.session.end()

# Close psycopg2 session.
db_conn.close()